### Importar librerias


In [46]:
from tokenize import triple_quoted
import requests 
requests.packages.urllib3.disable_warnings()
import pandas as pd
pd.options.mode.chained_assignment = None
import time
import dataframe_image as dfi
from PIL import Image
from datetime import datetime
from cim_to_id import cim_to_id as cmm
import random
import time
import dataframe_image as dfi
from PIL import Image
from datetime import datetime
from cim_to_id import cim_to_id as cmm
import random

### Definir parametros, opciones y fecha 

In [47]:

def format_float(value):
    return f'{value:,.2f}'

pd.options.display.float_format = format_float

currentDateAndTime = datetime.now()
currentTime = str(currentDateAndTime.strftime("%H:%M"))
hora = int(currentTime[0:2])
if hora <= 13:
    ventana = "1"
    cual = "## PRIMERA VENTANA ##"
elif hora >= 14:
    ventana = "2"
    cual = "## SEGUNDA VENTANA ##"
    
print("La hora actual es ",currentTime,"\n",cual,"\n")


La hora actual es  10:55 
 ## PRIMERA VENTANA ## 



### Token

In [48]:
now = str(datetime.now())
filename = ("Extracciones-"+now[0:10]+".png")
token = str(random.randint(111111111,99999999999))
print(f"Tu token es: {token}")
time.sleep(1)
print("Espere un segundo...")
time.sleep(0.5)

Tu token es: 29001618992
Espere un segundo...


## MOTOR DE EXTRACCIONES

### TEST BUSCAR LOP


### BUSCAR TODOS LOS ALYC CON SE

In [49]:
URL_GENERAL = "https://riskzone.anywhereportfolio.com.ar:9099/api/solicitudextraccion/"+token+"/getsolicitudextraccionbyalyc?CurrentPage=1&ItemsPerPage=100"
tokenobj = {'key': 'value'}
REQUEST_GENERAL = requests.get(URL_GENERAL, json = tokenobj)
data  = (REQUEST_GENERAL.json())
data = (data["Items"])
data = pd.DataFrame(data)
listaalycs = (data["MiembroCompensadorID"])
#ACA SE CREAN LOS DF A LOS QUE SE VA CONCATENANDO LA DATA
DATAFINAL = pd.DataFrame()
DATAFINALPORCIM = pd.DataFrame()

## LOOP PARA BUSCAR POR CIM Y LUEGO DENTRO DE LA INFO DE CADA CIM Y LUEGO POR FINALIDAD

In [50]:
for alyc in listaalycs:

    listacim = []
    aloc = str(alyc)
    URL_PORALYC = "https://riskzone.anywhereportfolio.com.ar:9099/api/solicitudextraccion/"+token+"/getsolicitudextraccionbycim?AlycID="+aloc
    #tokenobj = {'key': 'value'}
    try:
        PORCIM = requests.get(URL_PORALYC)#, json = tokenobj)
        data  = (PORCIM.json())
        df = pd.DataFrame(data)
    except:
        pass
    try:
        df_cim = df["CimID"][0]
        listacim.append(df_cim)
    except:
        pass
    #SEGUNDO LOOP
    for CIMID in listacim:
         URL_PORCIM = "https://riskzone.anywhereportfolio.com.ar:9099/api/solicitudextraccion/"+token+"/getsolicitudextraccionbyneteo?AlycID="+aloc+"&CimID="+str(CIMID)
         y = requests.get(URL_PORCIM, json = tokenobj)
         SE  = (y.json())
         SE = (pd.DataFrame(SE))
         DATAFINALPORCIM = pd.concat([DATAFINALPORCIM, SE])

         
         for ID in SE["CuentaNeteoID"]:
        #esto trae solo las que son finalidad 2 (margenes), hay que agregar que tamben traiga las de FCGIM
            URL_PORFINALIDAD2 = "https://riskzone.anywhereportfolio.com.ar:9099/api/solicitudextraccion/"+token+"/getsolicitudextraccionbymensajes?AlycID="+aloc+"&CimID="+str(CIMID)+"&neteoID="+str(ID)+"&finalidadID=2"
            finalidad2 = requests.get(URL_PORFINALIDAD2, json = tokenobj)
            PORFINALIDAD  = (finalidad2.json())
            PORFINALIDAD = (pd.DataFrame(PORFINALIDAD))
            DATAFINAL = pd.concat([DATAFINAL, PORFINALIDAD])
            URL_PORFINALIDAD9 = "https://riskzone.anywhereportfolio.com.ar:9099/api/solicitudextraccion/"+token+"/getsolicitudextraccionbymensajes?AlycID="+aloc+"&CimID="+str(CIMID)+"&neteoID="+str(ID)+"&finalidadID=9"
            finalidad9 = requests.get(URL_PORFINALIDAD9, json = tokenobj)
            PORFINALIDAD  = (finalidad9.json())
            PORFINALIDAD = (pd.DataFrame(PORFINALIDAD))
            DATAFINAL = pd.concat([DATAFINAL, PORFINALIDAD])



In [51]:
print(DATAFINAL.columns)


Index(['CuentaNeteoID', 'NeteoCodigo', 'FinalidadID', 'ExtraccionFecha',
       'ExtraccionHora', 'ActivoID', 'CodigoActivo', 'ActivoDescripcion',
       'Cantidad', 'Monto', 'Disponible', 'MargenDelDia', 'NoVerificado',
       'NoVerificadoTotal', 'SegundaVentana', 'TipoAprobacion'],
      dtype='object')


In [52]:

IDS = DATAFINALPORCIM[["NeteoCodigo","CuentaNeteoID","CimID","MiembroCompensadorID"]]
IDS = IDS.reset_index()
#IDS["Cuenta"] = IDS["Cuenta"].astype("str")


## OBTENER PROPIA

### obtener la cuenta propia

In [53]:
#cuentas = pd.read_csv("ListadoCIM.csv",index_col=0, encoding='latin-1')


In [54]:
#cuentas = cuentas[cuentas["Tipo Cuenta"] == "Propia"]
#cuentas =  cuentas.reset_index()
#cuentas = cuentas[["MC", "MC Cód.","Cód."]]
#cuentas = cuentas.rename(columns={"MC Cód.": "MC_COD", "Cód.": "MC_PROPIA_COD"})



In [55]:
url = "https://rm4api.testing.acsa:8099/api/cuentas/getcuentacompensacion"
df = requests.get(url, verify=False)
data_cuentas  = (df.json())
data_cuentas = pd.DataFrame(data_cuentas)



In [56]:
def get_cim_propia(MC):
    MC = str(MC)
    
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == MC]))
    propia = data_mc[data_mc["EsCuentaPropia"] == True]
    cim_codigo_propia = (propia["CuentaCompensacionCodigo"].to_string(name=False,dtype=False,index=False))
    return(int(cim_codigo_propia))


In [57]:
def get_mc_name(MC):
    MC = str(MC)
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == MC]))
    name_mc = data_mc["MiembroCompensadorDescripcion"].iloc[0] #.to_string(Name=False,dtype=False,index=False)
#cim_codigo_propia = (propia["CuentaCompensacionCodigo"].to_string(name=False,dtype=False,index=False))
    return(name_mc)


In [58]:
#def obtener_propia(MC_COD):
 #   linea = cuentas[cuentas["MC_COD"] == int(MC_COD)]
  #  return(int(linea["MC_PROPIA_COD"].item()))

#def obtener_nombre(MC_COD):
#    try:
#        linea = cuentas[cuentas["MC_COD"] == int(MC_COD)]
#        return(str(linea["MC"].item()))
#    except:
#        return(0)
#

### ACA LEE DE OTRO EXCEL LA PROPIA

In [59]:

DATAFINAL["PRIMER_ANALISIS"] = DATAFINAL["Disponible"] + DATAFINAL["Monto"]
DATAFINALPORCIM = DATAFINALPORCIM[[ "MiembroCompensadorID","CimID","CimCodigo", "CuentaNeteoID"]]
DATAFINAL = DATAFINAL.merge(DATAFINALPORCIM, on = "CuentaNeteoID", how = "left")
DATAFINAL = DATAFINAL.drop_duplicates()


In [60]:

#C_CIM = pd.read_excel(r"C:\Users\aggonzalez\Desktop\SE\MCCIM.xlsx")
#MC = MC_CIM[["CimCodigo", "MC_Cod"]]
#MC.dtypes.value_counts()

In [61]:

MC = data_cuentas[["CuentaCompensacionCodigo", "MiembroCompensadorCodigo"]]
MC = MC.rename(columns={"MiembroCompensadorCodigo":"MC_Cod","CuentaCompensacionCodigo":"CimCodigo"})
MC["MC_Cod"] = MC["MC_Cod"].astype(str).astype(int)
MC["CimCodigo"] = MC["CimCodigo"].astype(str).astype(int)


In [62]:
MC
#MC["CimCodigo"].dtypes.value_counts()

,CimCodigo,MC_Cod
0,2016,16
1,1016,16
2,2021,21
3,1021,21
4,1022,22
...,...,...
574,1187,187
575,1499,499
576,165590,499
577,165600,666


In [63]:

DATAFINAL["CimCodigo"] = DATAFINAL["CimCodigo"].astype(int)
cols = DATAFINAL.columns.tolist()
cols = cols[-1:] + cols[:-1]
DATAFINAL = DATAFINAL[cols]
DATAFINAL = DATAFINAL.merge(MC, on = "CimCodigo", how = "left")


### DEFINIR COLORES

In [64]:
def highlight_rows(row):
    value = row.loc["OUT"]
    if value == 0:
        color = '#FF0000' # Red
    elif value == 1:
        color = '#85e62c' # Green
    elif value == 2:
        color = '#ebff0a' # Otro green
    elif value == 3:
        color = "#209c05" #meio yellow
    elif value == 4:
        color = "#a8329b" #LILA BIEN FUERTE

    return ['background-color: {}'.format(color) for r in row]



In [65]:
### REORDENAR COLUMNAS
cols = DATAFINAL.columns.tolist()
cols = cols[-1:] + cols[:-1]
DATAFINAL = DATAFINAL[cols]


### PONER CUENTAS PROPIAS Y NOMBRES DE ALYC

In [66]:
cuentapropia = []
for i in range(len(DATAFINAL)):
    try:
        MC_COD = int(DATAFINAL["MC_Cod"][i])
        cimpropia = get_cim_propia(MC_COD)
    except:
        cimpropia = 0
    cuentapropia.append(cimpropia)

largo_maximo_nombre_alyc = 30


In [67]:

nombreMC = []
for i in range(len(DATAFINAL)):
    try:
        MC_COD = int(DATAFINAL["MC_Cod"][i])
        nombre_mc = get_mc_name(MC_COD)
        nombre_mc = str(nombre_mc[:largo_maximo_nombre_alyc])
    except: 
        nombre_mc = 0
    nombreMC.append(nombre_mc)



DATAFINAL["CuentaPropiaDelMC"] = cuentapropia
DATAFINAL["ALyC"] = nombreMC


In [68]:
lop = []
lopverificado = []
lopnoverificado = [] 
lopmargentotal = []

listasaldospropios = []
saldorealdelacuenta = []
neteodescrip = []
ingresoverificado = []
ingresonoverif=[]
cantidad=[]
margendeldia= []




In [69]:

def saldoreal(alyc, cim, neteo, finalidad):
    tokenobj = {"key":"value"}
    ENDPOINT = f"https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados?MCId={alyc}&CelID={cim}&NeteoID={neteo}&FinalidadID={finalidad}"
    q = requests.get(ENDPOINT, json = tokenobj)
    SALDOREAL  = (q.json())
    SALDOREAL = pd.DataFrame(SALDOREAL)
    SALDOREAL["SALDO INICIAL POSTA"] = SALDOREAL["Cotizacion"]*SALDOREAL["SaldoInicialMoneda"]
    SALDOREALPESOS = SALDOREAL["SALDO INICIAL POSTA"].sum()
    return(SALDOREALPESOS)

# ACA SE CONSIGUEN LOS SALDOS DE LA CUENTA PROPIA 


### ACA SE CONSIGUEN LOS SALDOS DE LA CUENTA PROPIA 


In [70]:
#cuentas = pd.read_excel(r"C:\Users\aggonzalez\Desktop\SE\obtener_cim_id.xlsx")
#archivocuentas = pd.read_excel(r"C:\Users\aggonzalez\Desktop\SE\TipoDeCuentaPorCim.xlsx")

#def cim_to_id(cim):
 #   try:
  #      id=cuentas[cuentas["CuentaCompensacionCodigo"]== int(cim)].iloc[0,0]
   #     return(id)
    #except:
     #   return(cim)

def cim_to_id(cim):

    cim = str(cim)

    cimcod = data_cuentas[data_cuentas["CuentaCompensacionCodigo"] == cim]
    cim = cimcod["CuentaCompensacionID"].to_string(index=False,dtype=False)
    return(cim)

In [71]:

for i in range(len(DATAFINAL)):
    try:
        ALYCID = (DATAFINAL.iloc[i,19])
        #print("alyc ID ", ALYCID)
        PROPA = (DATAFINAL.iloc[i,21])
        #print("PROPIA CIM COD ", PROPA)
        CIM = (DATAFINAL.iloc[i,20])
        #print("CIM ", CIM)
        NETEO = (DATAFINAL.iloc[i,2])
        #print("NETEO ",NETEO)
        FINALIDAD = (DATAFINAL.iloc[i,4])
        #print(FINALIDAD)
        print(PROPA)
        PROPA = cim_to_id(PROPA)
        print("ID ", PROPA)
        #print("PROPIA ID ", PROPA)

        #if PROPA == 162950:
        #    PROPA = 15380
        e = {'SaldoInicial':'sum', 'MargenRequeridoTotal':'sum'}

        tokenobj = {'key': 'value'}
        URLSALDO = f"https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados?MCId={ALYCID}&CelID={PROPA}"
        q = requests.get(URLSALDO, json = tokenobj)
        SALDOALYC  = (q.json())
        SALDOALYC = pd.DataFrame(SALDOALYC)



        saldoprevia = (SALDOALYC[['MiembroCompensadorID', 'CuentaCompensacionID', "MonedaDescripcion",'FinalidadID',"SaldoInicial","MargenRequeridoTotal"]]) #cambiar Anterior por Total
        saldoprevia["AP5"] = saldoprevia["SaldoInicial"] + saldoprevia["MargenRequeridoTotal"] #CAMBIAR ANTERIOR POR Total
        saldoprevia = saldoprevia.groupby("FinalidadID").agg(e)
        saldoprevia = saldoprevia.reset_index()

        #print("AGRUPADO QUEDA: ")
        #print(saldoprevia)

        #print("AHORA SOLO LA FINALIDAD CORRESPONDIENTE AL MENSAJE: ")
        saldoprevia = (saldoprevia[saldoprevia["FinalidadID"] == FINALIDAD])
        #print(saldoprevia)

        #print("AHORA LO NECESARIO PARA QUE QUEDE EL SALDO DE AP5")
        saldoprevia["AP5"] = saldoprevia["SaldoInicial"] + saldoprevia["MargenRequeridoTotal"] #CAMBIAR ANTERIOR POR Total

        #print(saldoprevia["AP5"])
        #print("AHORA SE SUMA LO DE  AP5")
        calditos = saldoprevia["AP5"].sum()

        #print(calditos)
        saldodelapropia = calditos




        ## GET LOPS !!!

        
        urllop = f"https://riskzone.anywhereportfolio.com.ar:9099/api/lopdrp/getlopalycdrill?MCId={ALYCID}&CurrentPage=1"
        tokenobj = {'key': 'value'}
        LOP = requests.get(urllop, json = tokenobj)
        data = LOP.json()
        data = (data["mcLopInfoList"])
        df = pd.DataFrame(data)

        LOPNOVERIFICADO = df.LopNoVerificado[0]
        LOPVERIFICADO = df.LopVerificado.values[0]
        LOPMARGENTOTAL = df.LopMargenTotal[0]
        LOP = df.Lop[0]
        df.LopEstaNotificado[0]


        
    except:
        saldodelapropia = 666


    listasaldospropios.append(saldodelapropia)
    lop.append(LOP)
    lopmargentotal.append(LOPMARGENTOTAL)
    lopverificado.append(LOPVERIFICADO)
    lopnoverificado.append(LOPNOVERIFICADO)


    


    tokenobj = {"key":"value"}
    ENDPOINT = f"https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados?MCId={ALYCID}&CelID={CIM}&NeteoID={NETEO}&FinalidadID={FINALIDAD}"
    H = requests.get(ENDPOINT, json = tokenobj)
    SALDOREAL  = (H.json())
    SALDOREAL = pd.DataFrame(SALDOREAL)


    SALDOREAL["MARGEN INICIAL POSTA"] = SALDOREAL["Cotizacion"]*SALDOREAL["MargenRequeridoAnterior"]
    SALDOREAL["SALDO INICIAL POSTA"] = SALDOREAL["Cotizacion"]*SALDOREAL["SaldoInicialMoneda"]
    SALDOREAL["SALDO INICIAL POSTA"] = SALDOREAL["SALDO INICIAL POSTA"]+SALDOREAL["MARGEN INICIAL POSTA"]
    INGRESOVERIFICADO = SALDOREAL["IngresoVerificado"].sum()
    SALDOREALPESOS = SALDOREAL["SALDO INICIAL POSTA"].sum()
    INGRESO_NO_VERIF = SALDOREAL["IngresoNoVerificado"].sum()



    MG2 = SALDOREAL["MargenRequeridoDelDia"].sum()


    cantidad.append(int(DATAFINAL["Cantidad"][i]))
    nombrecuenta = SALDOREAL["CuentaNeteoDescripcion"].iloc[0]
    neteodescrip.append(nombrecuenta)
    ingresoverificado.append(INGRESOVERIFICADO)
    saldoposta = SALDOREALPESOS
    saldorealdelacuenta.append(saldoposta)
    margendeldia.append(MG2)
    ingresonoverif.append(INGRESO_NO_VERIF)

    #SALDO INICIAL ESTA PESIFICADO, SALDOINICIAL MONEDA ESTA EN VALOR ORIGNIAL DIEZ





36421
ID  14839
55666
ID  14996
7319
ID  14887
27693
ID  14895


In [72]:
print(cim_to_id(101899))

15232


In [73]:

DATAFINAL["Saldo de la propia"] = listasaldospropios
DATAFINAL["Saldo POSTA"] = saldorealdelacuenta
DATAFINAL["Neteo Descripcion"] = neteodescrip
DATAFINAL["Ingresos Verificados"] = ingresoverificado
DATAFINAL["Margen Requerido Del Dia"] = margendeldia
DATAFINAL["NoVerificado2"] = ingresonoverif
DATAFINAL["Cantidad"]=cantidad
DATAFINAL["EstadoLOP"] = lop
DATAFINAL["LopNoVerificado"] = lopnoverificado
DATAFINAL["LopVerificado"] = lopverificado
DATAFINAL["LopMargenTotal"] = lopmargentotal


In [74]:
#DATAFINAL["Tipo de CIM"] = tipo_de_cuenta
if ventana == "1":
    DATAFINAL["PRIMER_ANALISIS"] = DATAFINAL["Saldo POSTA"] + DATAFINAL["Monto"] +DATAFINAL["Ingresos Verificados"]
elif ventana == "2":
    DATAFINAL["PRIMER_ANALISIS"] = DATAFINAL["Saldo POSTA"] + DATAFINAL["MargenDelDia"] + DATAFINAL["Monto"] +DATAFINAL["Ingresos Verificados"]


In [75]:

tipodeaprobacion = []
for i in range(len(DATAFINAL)):
    primeranalisis = int(DATAFINAL["PRIMER_ANALISIS"][i])
    SE = int(DATAFINAL["Monto"][i])
    saldoalyc =  int(DATAFINAL["Saldo de la propia"][i])
    noverificado = int(DATAFINAL["NoVerificado"][i])
    activo = DATAFINAL["ActivoDescripcion"][i]
    FCI = "FCI"

    

    if primeranalisis >= 0:
        aprobacion = 1
    else:
        if primeranalisis + saldoalyc >= 0:
            aprobacion = 3
            #print(primeranalisis + saldoalyc)
        else:
            if primeranalisis + noverificado >= 0:
                aprobacion = 2
            else:
                aprobacion = 0
    if ventana == str(2):
        if FCI in activo:
            aprobacion = 4
    elif ventana == str(1):
        pass

    tipodeaprobacion.append(aprobacion)


In [76]:

DATAFINAL["OUT"] = tipodeaprobacion
DATAFINAL["CimCodigo"] = DATAFINAL["CimCodigo"].astype(str)
DATAFINAL = DATAFINAL.sort_values(by = "CimCodigo")
IP = DATAFINAL[DATAFINAL["NoVerificado"] != 0]

#print(DATAFINAL.columns)
#RESHAPE DATAFRAME


### REFORMAR EL DATAFRAME

### ADJUNTAR LOS ID MERGEANDO EL DATAFRAME IDS

In [77]:
#DATAFINAL = DATAFINAL.merge(IDS, on = "NeteoCodigo", how = "left")


### PROBAR QUE ESTE TODO BIEN

In [78]:
print(DATAFINAL.columns)


Index(['MC_Cod', 'CimCodigo', 'CuentaNeteoID', 'NeteoCodigo', 'FinalidadID',
       'ExtraccionFecha', 'ExtraccionHora', 'ActivoID', 'CodigoActivo',
       'ActivoDescripcion', 'Cantidad', 'Monto', 'Disponible', 'MargenDelDia',
       'NoVerificado', 'NoVerificadoTotal', 'SegundaVentana', 'TipoAprobacion',
       'PRIMER_ANALISIS', 'MiembroCompensadorID', 'CimID', 'CuentaPropiaDelMC',
       'ALyC', 'Saldo de la propia', 'Saldo POSTA', 'Neteo Descripcion',
       'Ingresos Verificados', 'Margen Requerido Del Dia', 'NoVerificado2',
       'EstadoLOP', 'LopNoVerificado', 'LopVerificado', 'LopMargenTotal',
       'OUT'],
      dtype='object')


# DEFINIR EL DF FINAL

In [79]:
FINALIDAD = []
for i in range(len(DATAFINAL)):
    if DATAFINAL["FinalidadID"][i] == 2:
        fin = "Margenes"
    elif DATAFINAL["FinalidadID"][i] == 9:
        fin = "FGIMC"
    else:
        fin = "Otra(?)"
    FINALIDAD.append(fin)

DATAFINAL["FinalidadDescripcion"] = FINALIDAD
    

In [80]:
DATA2 =DATAFINAL
DATAFINAL = DATAFINAL[["ExtraccionHora","MC_Cod","ALyC","CimCodigo", "NeteoCodigo","Neteo Descripcion", "ActivoDescripcion","ActivoID","FinalidadID","FinalidadDescripcion","Cantidad","Monto", "Saldo POSTA","Ingresos Verificados", "NoVerificado", "MargenDelDia","PRIMER_ANALISIS","CuentaPropiaDelMC", "Saldo de la propia","OUT", "EstadoLOP","LopNoVerificado","LopVerificado","LopMargenTotal", "MiembroCompensadorID","CimID","CuentaNeteoID" ]]
DATAFINAL = DATAFINAL.rename(columns={"ExtraccionHora":"Hora","MC_Cod":"MC","CimCodigo": "CIM", "NeteoCodigo": "Cuenta","ActivoDescripcion":"Activo","Saldo POSTA":"Saldo Inicial","NoVerificado":"Ingreso No Verificado","MargenDelDia":"Margenes","PRIMER_ANALISIS":"Saldo Consolidado Final","CuentaPropiaDelMC":"Propia MC", "Saldo de la propia":"Saldo MC"})

#FORMATO NUMEROS


### FORMATEAR NUMEROS

In [81]:

DATAFINAL['Monto'] = DATAFINAL['Monto'].astype('int64')
DATAFINAL.loc[:, "Monto"] = DATAFINAL["Monto"].map('{:,}'.format)

DATAFINAL['Saldo Inicial'] = DATAFINAL['Saldo Inicial'].astype('int64')
DATAFINAL.loc[:, "Saldo Inicial"] = DATAFINAL["Saldo Inicial"].map('{:,}'.format)

DATAFINAL['Ingresos Verificados'] = DATAFINAL['Ingresos Verificados'].astype('int64').map('{:,}'.format)
DATAFINAL['Ingreso No Verificado'] = DATAFINAL['Ingreso No Verificado'].astype('int64').map('{:,}'.format)
DATAFINAL['Margenes'] = DATAFINAL['Margenes'].astype('int').map('{:,}'.format)
DATAFINAL['Saldo Consolidado Final'] = DATAFINAL['Saldo Consolidado Final'].astype('int64').map('{:,}'.format)

DATAFINAL['Saldo MC'] = DATAFINAL['Saldo MC'].astype('int64').map('{:,}'.format)
DATAFINAL['LopNoVerificado'] = DATAFINAL["LopNoVerificado"].astype('int64').map('{:,}'.format)
DATAFINAL['LopVerificado'] = DATAFINAL["LopVerificado"].astype('int64').map('{:,}'.format)
DATAFINAL["LopMargenTotal"] = DATAFINAL["LopMargenTotal"].astype('int64').map('{:,}'.format)

DATAFINAL['Cantidad'] = DATAFINAL['Cantidad'].astype('int64').map('{:,}'.format)
#DATAFINAL["FinalidadDescripcion"] = FinalidadDescripcion



### STYLEAR EL DATAFRAME, CREAR Y MOSTRAR IMAGEN

In [95]:

df_styled = DATAFINAL.reset_index(drop=True).style.apply(highlight_rows, axis=1)
#dfi.export_png(df_styled, filename,max_rows=-1,table_conversion='matplotlib')
df_styled.export_png(filename ,table_conversion='chrome')
IMAGEN = Image.open(filename)
IMAGEN.show()



### SOLICITUDES CON INFORME DE PAGO

In [83]:


apro = DATAFINAL[DATAFINAL["OUT"] == 2]
apro = apro.reset_index().drop_duplicates()

if len(apro)>0:
    print("\n### SOLICITUDES CON INFORME DE PAGO A VERIFICAR ### ")
    print(apro[["MC","CIM","Ingreso No Verificado"]])
else:
    print("\n## NO HAY SOLICITUDES DE EXTRACCIÓN QUE DEPENDAN DE INFORMES DE PAGO ##")


## NO HAY SOLICITUDES DE EXTRACCIÓN QUE DEPENDAN DE INFORMES DE PAGO ##


### VERIFICACIONES POSTERIORES 

In [84]:

DATAFINAL["Saldo Inicial"] = DATAFINAL["Saldo Inicial"].str.replace(",","").astype(float)
DATAFINAL["Monto"] = DATAFINAL["Monto"].str.replace(",","").astype(float)
DATAFINAL["Saldo MC"] = DATAFINAL["Saldo MC"].str.replace(",","").astype(float)

DATAFINAL1 = DATAFINAL[DATAFINAL["OUT"] == 1]

#print("\n #### VERIFICAR QUE EL TOTAL DE EXTRACCIONES PARA UNA MISMA CUENTA NO SUPERE EL SALDO DE LA MISMA ### \n")

d = {'Monto':'sum', 'Saldo Inicial':'first'}
check1 = DATAFINAL1.groupby("Cuenta").agg(d)
check1["Resultado"] = check1["Saldo Inicial"] - check1["Monto"]
check1 = check1[check1["Resultado"] < 0]

if len(check1) > 0:
    print(check1)
else:
    print("\n## NINGUNA CUENTA ESTA EXTRAYENDO MÁS DEL SALDO QUE POSEE ## ")

DATAFINAL3 = DATAFINAL[DATAFINAL["OUT"] == 3]


#print("\n #### VERIFICAR QUE EL TOTAL DE EXTRACCIONES CUBIERTOS CON EL SALDO DE LA PROPIA NO SUPERE EL SALDO DE LA MISMA ### \n")

e = {'Monto':'sum', 'Saldo MC':'first'}
check2 = DATAFINAL3.groupby("Propia MC").agg(e)
check2["Resultado"] = check2["Saldo MC"] - check2["Monto"] 
check2 = check2[check2["Resultado"] < 0]

if len(check2) > 0:
    print("\n## ATENCION - EXTRACCIONES SUPERAN SALDO DE LA PROPIA: \n")
    print(check2)
else:
    print("\n## NINGUNA CUENTA PROPIA ESTA CUBRIENDO MÁS DEL SALDO QUE POSEE ##\n ")







## NINGUNA CUENTA ESTA EXTRAYENDO MÁS DEL SALDO QUE POSEE ## 

## NINGUNA CUENTA PROPIA ESTA CUBRIENDO MÁS DEL SALDO QUE POSEE ##
 


In [85]:
sys.maxsize

9223372036854775807

## FORMATEO DEL JSON

In [86]:

DATA2 = DATA2[["ExtraccionHora","MC_Cod","ALyC","CimCodigo", "NeteoCodigo","Neteo Descripcion", "ActivoDescripcion","ActivoID","FinalidadID","FinalidadDescripcion","Cantidad","Monto", "Saldo POSTA","Ingresos Verificados", "NoVerificado", "MargenDelDia","PRIMER_ANALISIS","CuentaPropiaDelMC", "Saldo de la propia","OUT", "MiembroCompensadorID","CimID","CuentaNeteoID" ]]
DATA2 = DATA2.rename(columns={"OUT":"Resultado","ExtraccionHora":"ExtraccionHora","ALyC":"MiembroCompensadorDescripcion","MC_Cod":"MiembroCompensadorCodigo","CimCodigo": "CuentaCompensacionCodigo","Ingresos Verificados":"IngresosVerificados", "NeteoCodigo": "CuentaNeteoCodigo","Neteo Descripcion":"CuentaNeteoDescripcion","Saldo POSTA":"Saldo","NoVerificado":"IngresosNoVerificado","PRIMER_ANALISIS":"SaldoConsolidadoFinal","CuentaPropiaDelMC":"CuentaCompensacionCodigoPropia", "Saldo de la propia":"SaldoCuentaPropia","CimID":"CuentaCompensacionID"})
DATA2["CuentaCompensacionCodigoPropia"] = DATA2["CuentaCompensacionCodigoPropia"].astype("str")
DATA2["Resultado"] = DATA2["Resultado"].astype("str")


In [87]:
DATA2.to_json("json_records.json",orient="records")



In [88]:


def get_cim_propia(MC):
    MC = str(MC)
    url = "https://rm4api.testing.acsa:8099/api/cuentas/getcuentacompensacion"
    df = requests.get(url, verify=False)

    data_cuentas  = (df.json())
    data_cuentas = pd.DataFrame(data_cuentas)
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == MC]))
    propia = data_mc[data_mc["EsCuentaPropia"] == True]
    cim_codigo_propia = (propia["CuentaCompensacionCodigo"].to_string(name=False,dtype=False,index=False))
    return(cim_codigo_propia)

print(get_cim_propia(205))

4675
